In [11]:
import getpass
import os

os.environ["GROQ_API_KEY"] = "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"

In [12]:
%pip install -qU langchain-groq

In [13]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.2-3b-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [14]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Je aime le programmation.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 55, 'total_tokens': 62, 'completion_time': 0.004330176, 'prompt_time': 0.009206014, 'queue_time': 0.292379163, 'total_time': 0.01353619}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'stop', 'logprobs': None}, id='run-a0d1f835-9430-4d33-b25b-cf29edc2e1a1-0', usage_metadata={'input_tokens': 55, 'output_tokens': 7, 'total_tokens': 62})

**Identifying the stakeholders and end-users from the problem statement**

In [15]:
problem_statement = f"""NewYork Metro station wants to establish a TicketDistributor machine that issues tickets for passengers
travelling in metro rails. Travelers have options of selecting a ticket for a single trip, round trips or for
multiple trips. They can also issue a metro pass for regular passengers or a time card for a day, a week or
a month according to their requirements. The discounts on tickets will be provided to frequent travelling
passengers. The machine is also supposed to read the metro pass and time cards issued by the metro
counters or machine. The ticket rates differ based on whether the traveler is a child or an adult. The
machine is also required to recognize original as well as fake currency notes. The typical transaction consists of a user using the display interface to select the type and quantity of tickets and then choosing a
payment method of either cash, credit/debit card or smartcard. The ticket or tickets are printed and
dispensed to the user. Also the messaging facilities after every transaction are required on the registered
number. The system can also be operated comfortably by a touch-screen. A large number of heavy
components are to be used. We do not want our system to slow down, and also usability of the machine.
The TicketDistributor must be able to handle several exceptions, such as aborting the transaction for
incomplete transactions, insufficient amount given by the travelers to the machine, money return in case
of aborted transaction, change return after successful transaction, showing insufficient balance in the
card, updated information printed on the tickets e.g. departure time, date, time, price, valid from, valid
till, validity duration, ticket issued from and destination station. In case of exceptions, an error message is
to be displayed. We do not want user feedback after every development stage but after every two stages
to save time. The machine is required to work in a heavy load environment such that at the morning and
evening time in weekdays, and in weekends performance and efficiency would not get affected."""

co_analyst_context = """ You are a requirements analyst powered by AI along with a human analyst and you need to identify the stakeholders and end-users"""

prompt = """Assume you are a requirement analyst responsible for developing requirements for a MetroTicketDistributor
system.
{co_analyst_context}
Using the problem statement below, generate the key stakeholders and end-users.

Problem Statement:
{problem_statement}
 """

stakeholders = llm.invoke(prompt)
print(stakeholders.content)

{co_analyst_context}

As a requirement analyst responsible for developing requirements for a MetroTicketDistributor system, I have identified the key stakeholders and end-users for this project.

**Key Stakeholders:**

1. **Metro Officials**: Responsible for managing the metro system, including ticketing, operations, and maintenance.
2. **Transportation Department**: Oversees the overall transportation infrastructure, including the metro system.
3. **Ticketing System Providers**: Suppliers of ticketing systems for the metro, who may be involved in the implementation and maintenance of the system.
4. **System Integrators**: Companies responsible for integrating the new system with existing infrastructure and systems.
5. **Regulatory Bodies**: Government agencies responsible for ensuring compliance with regulations and standards related to ticketing and transportation systems.
6. **Customer Service Representatives**: Responsible for providing support and assistance to customers using the

**Eliciting requirements from the stakeholders using different elicitation techniques**

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a requirements coanalyst that uses elicitation techniques to get requirements from {stakeholders} and list those elicitation techniques and justify why and how the chosen elicitation techniques are helpful in gathering requirements.
            Do it in the following format :
            Elicitation technique name -->
            Why :
            How :
            """
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm

elicitation_tech = chain.invoke(
    {
     "stakeholders" : stakeholders,
     "input" : problem_statement,
    }
)

print(elicitation_tech.content)

Here are the elicitation techniques used to gather requirements for the MetroTicketDistributor system:

**1.** **Interviews -->**
Why : Interviews allow for in-depth discussions with stakeholders and end-users to gather detailed information about their needs and requirements.
How : Conduct one-on-one interviews with key stakeholders, including Metro Officials, Transportation Department, Ticketing System Providers, System Integrators, Regulatory Bodies, Customer Service Representatives, Metro Passengers, Ticketing System Administrators, Metro Staff, Transportation Workers, and System Administrators. Ask open-ended questions to gather information about their needs, requirements, and expectations from the system.

**2.** **Surveys -->**
Why : Surveys help to gather information from a larger group of people, including Metro Passengers, Ticketing System Administrators, Metro Staff, Transportation Workers, and System Administrators.
How : Create a survey questionnaire that covers the require

**Generate User Stories**

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a requirements coanalyst that uses elicitation techniques {elicitation_tech} to get all possible functionalities for each user in {stakeholders} in the form of user stories (both front and back of card). Therefore, list all possible user stories with the appropriate format.
            The front of the card will have the following format :
            As a [user role], I want to [goal] so that I can [reason].
            The back of the card will contain Success and Failure scenario.
            Use {example} for format.
             """
        ),
        ("human", "{input}"),
    ]
)

example = """Front of the Card :
As a registered	user I want to log in so that I	can	access subscriber‐only content
Back of the	Card :
1. Success - valid user logged in and referred to home page
(a) 'Remember me' ticked-store cookie/ automatic login next
time.
(b) 'Remember me' not ticked - force login next time.
2.   Failure-display message
(a) "Email address in wrong format"
(b) "Unrecognized user name, please try again"
(c) "Incorrect password, please try again"
(d) "Service unavailable, please try again"
(e) Account has expired - refer to account renewal sales page.
"""

chain = prompt | llm

generated_user_stories = chain.invoke(
    {
     "elicitation_tech" : elicitation_tech,
     "stakeholders" : stakeholders,
     "example" : example,
     "input" : problem_statement,
    }
)

print(generated_user_stories.content)

Here are the user stories for the TicketDistributor machine:

**Front of the Card**

1. As a registered user, I want to select a ticket type (single trip, round trip, multiple trips) so that I can choose my travel option.
	* Back of the Card:
		+ Success:
			- (a) Ticket type selected successfully
			- (b) Ticket type validation (e.g. single trip, round trip, multiple trips)
		+ Failure:
			- (a) Invalid ticket type selected
			- (b) Ticket type not available
			- (c) Error message displayed
2. As a registered user, I want to select the quantity of tickets so that I can choose the number of tickets I need.
	* Back of the Card:
		+ Success:
			- (a) Quantity selected successfully
			- (b) Quantity validation (e.g. positive integer)
		+ Failure:
			- (a) Invalid quantity selected
			- (b) Quantity not available
			- (c) Error message displayed
3. As a registered user, I want to select the payment method (cash, credit/debit card, smartcard) so that I can pay for my tickets.
	* Back of the

**Validate User Stories (according to INVEST framework)**

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a requirements coanalyst that uses the INVEST framework to validate the {generated_user_stories} i.e., make sure that the user stories follow the INVEST framework.
            The INVEST framework rules are :
1. All the stories that have been broken down are mostly independent	, hence making sure that they can be parallely developed.
2. Not every minute detail has been included in the cards, encouraging developers to seek further clarification and ensuring negotiability.
3. Stories have been written in regular business language that is equally valuable to both stakeholders and developers.
4. It is assumed that all the user stories mentioned below can be converted into working pieces of code within the working period of 1 to 3 weeks and hence are estimable	and small.
5. It is also written in a way where the completion of the story and its working is quantifiable and can be tested, hence being testable.
Give the validated user stories with this format :
The front of the card will have the following format :
            As a [user role], I want to [goal] so that I can [reason].
            The back of the card will contain Success and Failure scenario.
            Refer to {example}.
             """
        ),
        ("human", "{input}"),
    ]
)

example = """Front of the Card :
As a registered	user I want to log in so that I	can	access subscriber‐only content
Back of the	Card :
1. Success - valid user logged in and referred to home page
(a) 'Remember me' ticked-store cookie/ automatic login next
time.
(b) 'Remember me' not ticked - force login next time.
2.   Failure-display message
(a) "Email address in wrong format"
(b) "Unrecognized user name, please try again"
(c) "Incorrect password, please try again"
(d) "Service unavailable, please try again"
(e) Account has expired - refer to account renewal sales page.
"""

chain = prompt | llm

validated_user_stories = chain.invoke(
    {
        "generated_user_stories" : generated_user_stories,

     "stakeholders" : stakeholders,
     "example" : example,
     "input" : problem_statement,
    }
)

print(validated_user_stories.content)

Here are the validated user stories for the TicketDistributor machine using the INVEST framework:

**User Stories for Independent Development**

1. As a registered user, I want to select a ticket type (single trip, round trip, multiple trips) so that I can choose my travel option.
	* Back of the Card:
		+ Success:
			- (a) Ticket type selected successfully
			- (b) Ticket type validation (e.g. single trip, round trip, multiple trips)
		+ Failure:
			- (a) Invalid ticket type selected
			- (b) Ticket type not available
			- (c) Error message displayed
2. As a registered user, I want to select the quantity of tickets so that I can choose the number of tickets I need.
	* Back of the Card:
		+ Success:
			- (a) Quantity selected successfully
			- (b) Quantity validation (e.g. positive integer)
		+ Failure:
			- (a) Invalid quantity selected
			- (b) Quantity not available
			- (c) Error message displayed
3. As a registered user, I want to select the payment method (cash, credit/debit card,

**Prioritizing User Stories using MoSCoW**

In [20]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a requirements coanalyst that uses the MoSCoW technique (that is classifying user story into must have, should have, could have, won't have) to prioritize the validated user stories {validated_user_stories} taking into consideration the non-functional aspects as well.
            Display the user stories along with their MoSCoW counterparts, specify how, and provide details.
             """
        ),
        ("human", "{input}"),
    ]
)



chain = prompt | llm

prioritized_user_stories = chain.invoke(
    {
     "validated_user_stories" : validated_user_stories,
     "input" : problem_statement,
    }
)

print(prioritized_user_stories.content)

Based on the provided requirements, I will categorize the user stories into MoSCoW counterparts.

**Must-Have (M)**

1. As a registered user, I want to select a ticket type (single trip, round trip, multiple trips) so that I can choose my travel option.
	* Back of the Card:
		+ Success: (a) Ticket type selected successfully
		+ Failure: (a) Invalid ticket type selected
		+ (b) Ticket type not available
		+ (c) Error message displayed
2. As a registered user, I want to select the quantity of tickets so that I can choose the number of tickets I need.
	* Back of the Card:
		+ Success: (a) Quantity selected successfully
		+ Failure: (a) Invalid quantity selected
		+ (b) Quantity not available
		+ (c) Error message displayed
3. As a registered user, I want to select the payment method (cash, credit/debit card, smartcard) so that I can pay for my tickets.
	* Back of the Card:
		+ Success: (a) Payment method selected successfully
		+ Failure: (a) Invalid payment method selected
		+ (b) Paymen

**Identifying EPICs**

In [21]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Identify three different EPICs (or collection of user stories) from {validated_user_stories}, where conflicts between the requirements occur. These EPICs will have user stories that have conflicting interests. Describe these conflicts. If the conflicts can be resolved, state solutions.
             """
        ),
        ("human", "{input}"),
    ]
)



chain = prompt | llm

epics = chain.invoke(
    {
     "validated_user_stories" : validated_user_stories,
     "input" : problem_statement,
    }
)

print(epics.content)

Based on the provided requirements, I have identified three EPICs (or collection of user stories) that seem to have conflicting interests:

**EPIC 1: User Experience and Usability**

* User Stories:
	+ As a registered user, I want the system to be operated comfortably by a touch-screen so that I can easily interact with the system.
	+ As a registered user, I want the system to handle heavy loads without slowing down so that I can use the system efficiently.
	+ As a registered user, I want the system to be operated comfortably by a touch-screen so that I can easily interact with the system.
* Conflicts:
	+ The first two user stories require the system to handle heavy loads without slowing down, which may conflict with the requirement to operate comfortably by a touch-screen.
	+ The system may need to optimize its performance to handle heavy loads while still providing a smooth user experience.

**EPIC 2: Payment and Transaction**

* User Stories:
	+ As a registered user, I want to selec